# Implementing motion detection to find Kodak in a video frame
Once i have code that can detect Kodak in a video, I'll connect it to the camera. 

In [1]:
# Importing packages
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import pandas as pd
import cv2

import subprocess
import IPython.display as ipd

import torch
from torchvision import models
import torch.nn as nn

In [2]:
test1_input = "test1.MOV"

In [3]:
# Converting from .mov to .mp4 and removing the last second of the video
subprocess.run(['ffmpeg', '-i', test1_input, '-qscale', '0', 'test1.mp4', '-loglevel', 'quiet', ])

CompletedProcess(args=['ffmpeg', '-i', 'test1.MOV', '-qscale', '0', 'test1.mp4', '-loglevel', 'quiet'], returncode=0)

In [ ]:
# Seeing the video
ipd.Video('test1.mp4', width = 300)

## Ok, lets do the motion detector now
If a moving object appears in 5 consecutive frames, I want the box to be passed to my pre-trained kodak_detector.

In [3]:
# Function to normalize the box before sending it to my model
def preprocess_frame(frame):
    resized_frame = cv2.resize(frame, (224, 224))
    normalized_frame = resized_frame / 255
    processed = np.expand_dims(normalized_frame, axis = 0)
    return processed

def is_it_kodak(frame):
    processed_frame = preprocess_frame(frame)
    prediction = model.predict(processed_frame)
    pos_or_neg = np.argmax(prediction, axis = 1)[0] == 1
    return pos_or_neg

In [4]:
# Preparing for the video
class_names = [0, 1]
background = None
motion_frames = 0
consecutive_frames = 5

In [5]:
# Defining model architecture
model = models.resnet18()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

In [7]:
# Loading model weights
model.load_state_dict(torch.load('is_that_kodak.pth'))

<All keys matched successfully>

In [8]:
test1 = cv2.VideoCapture('test1.mp4')

In [9]:
while True:
    # Take in first frame, convert it to gray scale, then blur it
    check, frame = test1.read()
    if not check:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (41, 41), 0)

    # If there is not yet a background, make one
    if background is None:
        background = gray
        continue
    
    # Finding the difference between background and current frame
    diff_frame = cv2.absdiff(background, gray)

    # If the difference is greater than 50, make a binary mask where the motion is white
    threshold = cv2.threshold(diff_frame, 100, 255, cv2.THRESH_BINARY)[1]
    threshold = cv2.dilate(threshold, None, iterations = 2)

    # Given motion, find the contours of the object
    contours, _ = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    motion_detected = False

    # Making sure the object isn't too big
    for contour in contours:
        if cv2.contourArea(contour) < 10000:
            continue
        motion = 1
        (x, y, w, h) = cv2.boundingRect(contour)
        # Drawing a box around the moving object
        cv2.rectangle(frame, (x - 20, y - 20), (x + w + 20, y + h + 20), (0, 0, 255), 3)
        motion_detected = True
    
    if motion_detected:
        motion_frames += 1
    else:
        motion_frames = 0

    # If motion is detected in 5 consecutive frames, save the frame and send to model
    if motion_frames >= consecutive_frames:
        frame_to_identify = frame[y:y + h, x:x + w]
        if is_it_kodak(frame_to_identify):
            print("CAT ALERT OMG")
            cv2.imwrite('detected_cat.jpeg', frame_to_identify)
        motion_frames = 0
    
    # Displaying image in gray_scale 
    cv2.imshow("Gray Frame", gray) 
  
    # Displaying the difference in currentframe to 
    # the staticframe(very first_frame) 
    cv2.imshow("Difference Frame", diff_frame) 
  
    # Displaying the black and white image in which if 
    # intensity difference greater than 30 it will appear white 
    cv2.imshow("Threshold Frame", threshold) 
  
    # Displaying color frame with contour of motion of object 
    cv2.imshow("Color Frame", frame) 
  
    key = cv2.waitKey(1) 
    # if q entered whole process will stop 
    if key == ord('q'): 
        break

AttributeError: 'ResNet' object has no attribute 'predict'

In [7]:
test1.release()
cv2.destroyAllWindows()